In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline 

path_to_data = 'data/clean/'

w_df = pd.read_csv(path_to_data +'works_clean.csv')

#### Create an empty df with composerName ('last name, first name'), birthYear, deathYear

In [2]:
w = w_df.composerName.value_counts()

comp_df = w.to_frame("birthYear")

comp_df['birthYear'] = ""
comp_df["deathYear"] = ""

comp_df[:10]

,birthYear,deathYear
"Beethoven, Ludwig van",,
"Wagner, Richard",,
"Tchaikovsky, Pyotr Ilyich",,
"Mozart, Wolfgang Amadeus",,
"Brahms, Johannes",,
"Strauss, Richard",,
"Bach, Johann Sebastian",,
"Mendelssohn, Felix",,
"Dvorak, Antonín",,
"Schubert, Franz",,


### Use Wikipedia API to determine composers birthYear and deathYear

In [3]:
import wikipediaapi
import re

wiki_wiki = wikipediaapi.Wikipedia('en')

In [40]:
not_found = [] #list of not found composers

for i in range(793, len(comp_df)): #request times out occasionally, so picking up where it stopped
    
    # transforming from 'Beethoven, Ludwig van' to 'Ludwig van Beethoven'
    composer = comp_df.index[i]
    names = re.split(r'[,]', composer) 
    names.reverse() 
    composer = " ".join(names)
    
    #some composers have alternate spellings in their name between []. Remove them to facilitate Wikipedia lookup.
    if "[" in composer:
        i_i = composer.find('[') -1
        j_j = composer.find(']') + 1
        composer = composer.replace(composer[i_i:j_j], "")
    
    comp_pg = wiki_wiki.page(composer)
    
    if comp_pg.exists():
        summary = comp_pg.summary
        
        birth_year = "0000"
        death_year = "9999"
        
        for word in re.split(r'\W+', summary): #get only non-words. 

            if word.isdigit() and len(word) == 4:

                if birth_year == "0000": #first occurrence
                    birth_year = int(word)
                    continue

                if death_year == "9999": #second occurrence
                    death_year = int(word)

        comp_df["birthYear"][i] = float(birth_year)
        comp_df["deathYear"][i] = float(death_year)

    else:
        not_found.append(composer)
        comp_df["birthYear"][i] = float('nan')
        comp_df["deathYear"][i] = float('nan')

#### Saving to file

In [44]:
comp_df.to_csv(path_to_data + "comp_dates.csv")

f = open(path_to_data + "notfound.txt","w+")

for comp in not_found:
    f.write(comp + "\n")
    
f.close()